In [1]:
import pickle
with open('test1.pkl', 'rb') as handle:
    (kmeans,visual_word,digital_image)= pickle.load(handle)
with open('test2.pkl','rb') as handles:
    (labels) = pickle.load(handles)

In [2]:
def build_histogram(descriptor_list, cluster_alg):
    histogram = np.zeros(len(cluster_alg.cluster_centers_))
    cluster_result =  cluster_alg.predict(descriptor_list)
    for i in cluster_result:
        histogram[i] += 1.0
    return histogram

In [3]:
def extract_denseSIFT(img):
    DSIFT_STEP_SIZE = 2
    sift = cv2.xfeatures2d.SIFT_create()
    disft_step_size = DSIFT_STEP_SIZE
    keypoints = [cv2.KeyPoint(x, y, disft_step_size)
            for y in range(0, img.shape[0], disft_step_size)
                for x in range(0, img.shape[1], disft_step_size)]
    descriptors = sift.compute(img, keypoints)[1]
#     gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors


# form histogram with Spatial Pyramid Matching upto level L with codebook kmeans and k codewords
def getImageFeaturesSPM(L, img, kmeans, k):
    W = img.shape[1]
    H = img.shape[0]   
    h = []
    for l in range(L+1):
        w_step = math.floor(W/(2**l))
        h_step = math.floor(H/(2**l))
        x, y = 0, 0
        for i in range(1,2**l + 1):
            x = 0
            for j in range(1, 2**l + 1):                
                des = extract_denseSIFT(img[y:y+h_step, x:x+w_step])                
                histo = build_histogram(des,kmeans)
                weight = 2**(l-L)
                h.append(weight*histo)
                #print(len(h))
                x = x + w_step
            y = y + h_step
            
    hist = np.array(h).ravel()
    # normalize hist
    dev = np.std(hist)
    hist -= np.mean(hist)
    hist /= dev
    return hist

In [31]:
len(des_list)

109755

In [4]:
import datetime
start = datetime.datetime.now()
count = 0
import math
import numpy as np
import cv2
import os
image_path_list = []
sift = cv2.xfeatures2d.SIFT_create()
for label in os.listdir("BOW"):
    image_path_list.append(os.path.join("BOW", label))
preprocessed_image = []
for path in image_path_list:
    for file_name in os.listdir(path):
        img = cv2.imread(os.path.join(path,file_name))
        histogram = getImageFeaturesSPM(1,img,kmeans,500)
        #print(count)
        preprocessed_image.append(histogram)
        #count += 1
end = datetime.datetime.now()
print(end-start)

0:16:56.035598


In [7]:
len(preprocessed_image[0])

1250

+ level 0-500d = len(visual_word)
+ level 1-2500d
+ level 2-10500d
+ Vector được extract cuối cùng của level đó sẽ bao gồm cả vector của level trước đó.
time: lv0: 0:04:43.394424 lv1: 0:09:05.957715

In [6]:
preprocessed_image[1]

array([-0.56420845, -0.57796964, -0.81190973, -0.66053673,  0.605492  ,
       -0.68805909, -0.26146245, -0.79814854,  1.01832745, -0.77062618,
       -0.26146245, -0.454119  , -0.74310382, -0.44035782, -0.42659664,
       -0.01376118, -0.49540254, -0.70182027, -0.35779073,  1.91280427,
        0.74310382,  0.44035782, -0.23394009,  0.61925318, -0.28898482,
       -0.52292491, -0.77062618,  0.27522364,  3.784325  , -0.63301436,
       -0.74310382,  0.77062618, -0.27522364, -0.35779073, -0.19265655,
        0.35779073,  0.04128355, -0.72934264, -0.82567091,  0.605492  ,
       -0.605492  , -0.61925318, -0.19265655,  1.63758063,  0.26146245,
        1.92656545,  0.74310382, -0.605492  ,  0.04128355, -0.74310382,
       -0.52292491, -0.46788018, -0.77062618, -0.55044727,  0.09632827,
       -0.72934264, -0.41283545, -0.72934264,  0.64677554, -0.09632827,
       -0.50916373,  1.10089454, -0.16513418, -0.756865  , -0.50916373,
       -0.55044727, -0.27522364, -0.71558145, -0.82567091, -0.20

In [7]:
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(preprocessed_image,labels, test_size=0.2, random_state=0)

clf = svm.SVC(kernel='linear')
clf.fit(X_train,y_train)
y_predict = clf.predict(X_test)
print("Accuracy2 %.2f %%" %(100*accuracy_score(y_test, y_predict)))

Accuracy2 77.43 %


In [19]:
from sklearn import neighbors
from sklearn.metrics import accuracy_score

clf = neighbors.KNeighborsClassifier(n_neighbors = 10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print("Accuracy1 %.2f %%" %(100*accuracy_score(y_test, y_pred)))


Accuracy1 69.91 %


(Ảnh đen trắng)
với visual word = 500:
+ BOW KNN: 60,26%,SVM: 70,21%
+ SVM level 0: 77.43% level 1:82.30% level 2: 84,07%
+ KNN level 0: 65.49% level 1:69.91% level 2: 75,22%

với visual word = 250:
+ BOW KNN: 59,76%,SVM: 66,57%
+ SVM level 0:78.32% level 1:83.63% level 2: 84.96%
+ KNN level 0:69.03% level 1:71.68% level 2: 76,55%

In [30]:
# import os
# import cv2
# import math
# image_path_list = []
# sift = cv2.xfeatures2d.SIFT_create()
# for label in os.listdir("BOW"):
#     image_path_list.append(os.path.join("BOW", label))
# des_list = []
# des_list1 = []
# labels = []
# count = 0
# for path in image_path_list:
#     for file_name in os.listdir(path):
#         #print(file_name)
#         count = count +1
#         img = cv2.imread(os.path.join(path,file_name),2)
#         #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#         kp,des = sift.detectAndCompute(img,None)
#         des_list1.append(des)
#         labels.append(path)
#         for i in range(len(des)):
#             if i%5 == 0: 
#                 des_list.append(des[i])
# #des_list = np.asarray(des_list)
# #des_list = np.concatenate(des_list,axis=0)
# count

1126

In [5]:
with open('SPM_feature.pkl', 'wb') as handle:
    pickle.dump((preprocessed_image), handle)